In [3]:
import numpy as np
import matplotlib.pyplot as plt
import uproot
import awkward as ak
import pandas as pd
import pylhe
from tqdm import tqdm
import itertools

In [4]:
def f(deltaeta, deltaphi):
    return np.sqrt(deltaeta**2 + deltaphi**2)
def function(f_Att, cs, k_factor):
    # Define Tree
    tree_electron_Att = f_Att['Delphes;1/Electron']
    tree_muon_Att = f_Att['Delphes;1/Muon']
    tree_jet_Att = f_Att['Delphes;1/Jet']
    tree_Emiss_Att = f_Att['Delphes;1/MissingET']
    
    # Define Branches
    Att_electron = tree_electron_Att.arrays(['Electron.Charge',
                                            'Electron.PT',
                                            'Electron.Eta',
                                            'Electron.Phi'], library='ak')
    Att_muon = tree_muon_Att.arrays(['Muon.Charge',
                                    'Muon.PT',
                                    'Muon.Eta',
                                    'Muon.Phi'], library='ak')
    Att_jet = tree_jet_Att.arrays(['Jet.Mass',
                                   'Jet.BTag',
                                   'Jet.PT',
                                   'Jet.Eta', 
                                   'Jet.Phi'], library='ak')
    Att_Emiss = tree_Emiss_Att.arrays(['MissingET.MET',
                                       'MissingET.Phi'], library='ak')
    
    # Define Variables
    EventSize = len(Att_jet['Jet.Mass'])
    jet_sort_pt = ak.sort(Att_jet['Jet.PT'], axis=-1, ascending=False)
    jet_num = ak.num(Att_jet['Jet.PT'], axis=-1)
    lepton_sort_pt = ak.sort(ak.concatenate([Att_electron['Electron.PT'], Att_muon['Muon.PT']], axis=-1), axis=-1, ascending=False)
    lepton_charge = ak.Array.tolist(ak.concatenate([Att_electron['Electron.Charge'], Att_muon['Muon.Charge']], axis=-1))
    lepton_eta = ak.concatenate([Att_electron['Electron.Eta'], Att_muon['Muon.Eta']], axis=-1)
    lepton_phi = ak.concatenate([Att_electron['Electron.Phi'], Att_muon['Muon.Phi']], axis=-1)
    bjet_pt = Att_jet['Jet.PT']*Att_jet['Jet.BTag']
    bjet_eta = Att_jet['Jet.Eta']*Att_jet['Jet.BTag']
    bjet_phi = Att_jet['Jet.Phi']*Att_jet['Jet.BTag']
    three_lepton = ak.where(ak.num(lepton_sort_pt)>=3)
    
    # At least three leptons and three b-jets
    event_signal = []
    for i in tqdm(three_lepton[0]): #Awkward Array has a length of 1
        b_pt = bjet_pt[i]
        num_bjet = len(b_pt[ak.where(b_pt)])
        if num_bjet >= 3:
            event_signal.append(i)
        
    # Transverse Momentum of leading and subleading lepton & Eta for Leptons
    event_lep = []
    for i in tqdm(event_signal):
        lep_eta = lepton_eta[i]
        lep_where = ak.where(np.abs(lep_eta)<2.5, 0, lep_eta)
        if lepton_sort_pt[i][2] > 25 and ak.count_nonzero(lep_where) == 0:
            event_lep.append(i)
        else:
            continue
    
    # Transverse Momentum of leading and subleading b-jet & Eta for b-jets
    event_jet = []
    for i in tqdm(event_lep):
        b_pt, b_eta  = bjet_pt[i], bjet_eta[i]
        b_pt = b_pt[ak.where(b_pt)]
        b_pt = ak.sort(b_pt, ascending=False)
        bjet_where = ak.where(np.abs(b_eta)<2.5, 0, b_eta)
        if b_pt[2] > 20 and ak.count_nonzero(bjet_where) == 0:
            event_jet.append(i)
        else:
            continue
    
    # Emiss
    event_emiss = []
    for i in tqdm(event_jet):
        if Att_Emiss['MissingET.MET'][i] > 30:
            event_emiss.append(i)
        else:
            continue
 
    # delta R between b-jets and leptons, leptons and leptons, b-jets and b-jets
    event_deltaR_bl = []
    for i in tqdm(event_emiss):
        lep_eta, lep_phi, b_eta, b_phi = lepton_eta[i], lepton_phi[i], bjet_eta[i], bjet_phi[i]
        b_eta, b_phi = b_eta[ak.where(b_eta)], b_phi[ak.where(b_phi)]

        deltaeta_bl = np.array([i-j for i in b_eta for j in lep_eta])
        deltaphi_bl = np.abs([i-j for i in b_phi for j in lep_phi])
        deltaphi_bl = np.array([2*np.pi-i if i > np.pi else i for i in deltaphi_bl])
    
        if min(f(deltaeta_bl, deltaphi_bl)) > 0.4:
            event_deltaR_bl.append(i)
    
    event_deltaR_bb = []
    for i in tqdm(event_deltaR_bl):
        b_eta, b_phi = bjet_eta[i], bjet_phi[i]
        b_eta, b_phi = b_eta[ak.where(b_eta)], b_phi[ak.where(b_phi)]
        
        m = b_eta - b_eta[:, np.newaxis]
        deltaeta_bb = m[np.triu_indices(len(m[0]), k = 1)]
        m = b_phi - b_phi[:, np.newaxis]
        deltaphi_bb = np.abs(m[np.triu_indices(len(m[0]), k = 1)])
        deltaphi_bb = np.array([2*np.pi-i if i > np.pi else i for i in deltaphi_bb])
        
        if min(f(deltaeta_bb, deltaphi_bb)) > 0.4:
            event_deltaR_bb.append(i)
            
    event_deltaR_ll = []
    for i in tqdm(event_deltaR_bb):
        lep_eta, lep_phi = lepton_eta[i], lepton_phi[i]
        m = lep_eta - lep_eta[:, np.newaxis]
        deltaeta_ll = m[np.triu_indices(len(m[0]), k = 1)]
        m = lep_phi - lep_phi[:, np.newaxis]
        deltaphi_ll = np.abs(m[np.triu_indices(len(m[0]), k = 1)])
        deltaphi_ll = np.array([2*np.pi-i if i > np.pi else i for i in deltaphi_ll])

        if min(f(deltaeta_ll, deltaphi_ll)) > 0.4:
            event_deltaR_ll.append(i)
    
    # HT of 2 leading leptons and three leading jets
    event_ht = []
    for i in tqdm(event_deltaR_ll):
        lep_pt, jet_pt = lepton_sort_pt[i], jet_sort_pt[i]
        ht = np.sum([lep_pt[0], lep_pt[1], lep_pt[2], jet_pt[0], jet_pt[1], jet_pt[2]])
        if ht > 320:
            event_ht.append(i)
                
    
    cs_b = cs * 1000
    sel_eff = len(event_ht)/EventSize
    uncertainty = np.sqrt(sel_eff * (1-sel_eff)/ EventSize)
    cs_f = cs_b * len(event_ht)/EventSize
    cs_ff = cs_f * k_factor
                
    return [[EventSize, len(event_signal), len(event_lep), len(event_jet), len(event_emiss), len(event_deltaR_bl), len(event_deltaR_bb), len(event_deltaR_ll), len(event_ht)], [cs_b, sel_eff, cs_f, uncertainty, cs_ff]]

In [19]:
y1 = uproot.open('../root/MG5_aMC_v2_7_3/ttW/Events/run_04/tag_1_delphes_events.root')
y2 = uproot.open('../root/MG5_aMC_v2_7_3/ttZ/Events/run_03/tag_1_delphes_events.root')
y3 = uproot.open('../root/MG5_aMC_v2_7_3/tttt/Events/run_01/tag_1_delphes_events.root')
ttW = function(y1, 0.004573, 1.35)
ttZ = function(y2, 0.01687, 1.56)
tttt = function (y3, 0.0006133, 2.04)

100%|██████████| 1492/1492 [00:00<00:00, 1869.77it/s]


In [22]:
y4 = uproot.open('../root/MG5_aMC_v2_7_3/ttW1/Events/run_01/tag_1_delphes_events.root')
y5 = uproot.open('../root/MG5_aMC_v2_7_3/ttZ1/Events/run_01/tag_1_delphes_events.root')
ttW1 = function(y4, 0.003944, 1.35)
ttZ1 = function(y5, 0.01293, 1.56)

100%|██████████| 288/288 [00:00<00:00, 1419.80it/s]


In [23]:
name = [r'Input Event Size',
        '3b3l Signature',
        r'Jet Selection',
        r'lepton Selection',
        r'$E_{T}^{miss}$ > 30GeV',
        r'Δ$R_{bl}$ > 0.4',
        r'Δ$R_{bb}$ > 0.4',
        r'Δ$R_{ll}$ > 0.4',
        r'$H_{T}$ Cut']
fd = pd.DataFrame({'Selection Cut (3b3l)': name,
                   'ttZ+j': ttZ[0],
                   '4t': tttt[0],
                   'ttW+j': ttW[0],
                   'ttZ': ttZ1[0],
                   'ttW': ttW1[0]
                   })
fd

,Selection Cut (3b3l),ttZ+j,4t,ttW+j,ttZ,ttW
0,Input Event Size,50000,50000,50000,50000,10000
1,3b3l Signature,481,2717,153,594,20
2,Jet Selection,315,1748,109,399,12
3,lepton Selection,302,1704,100,369,12
4,$E_{T}^{miss}$ > 30GeV,254,1592,95,307,10
5,Δ$R_{bl}$ > 0.4,254,1592,95,307,10
6,Δ$R_{bb}$ > 0.4,254,1592,95,307,10
7,Δ$R_{ll}$ > 0.4,234,1492,89,288,9
8,$H_{T}$ Cut,232,1485,86,281,7


In [24]:
name2 = ['Cross Section from MadGraph Generation (fb)',
         'Selection Efficiency',
         'Cross Section after Selection Cut (fb)',
         'Uncertainty',
         'NLO Cross Section after timing its k factor']
fd = pd.DataFrame({'Selection Cut (3b3l)': name2,
                   'ttZ+j': ttZ[1],
                   'tttt': tttt[1],
                   'ttW+j': ttW[1],
                   'ttZ': ttZ1[1],
                   'ttW': ttW1[1]
                   })
fd

,Selection Cut (3b3l),ttZ+j,tttt,ttW+j,ttZ,ttW
0,Cross Section from MadGraph Generation (fb),16.870000,0.613300,4.573000,12.930000,3.944000
1,Selection Efficiency,0.004640,0.029700,0.001720,0.005620,0.000700
2,Cross Section after Selection Cut (fb),0.078277,0.018215,0.007866,0.072667,0.002761
3,Uncertainty,0.000304,0.000759,0.000185,0.000334,0.000264
4,NLO Cross Section after timing its k factor,0.122112,0.037159,0.010619,0.113360,0.003727


In [34]:
name = [
     'ttZ',
     '4t',
     'ttW',
]
n1 = [0.135, 0.105, 0.011]
n2 = [ttZ[-1][-1], tttt[-1][-1], ttW[-1][-1]]
n3 = [ttZ1[-1][-1], tttt[-1][-1], ttW1[-1][-1]]
uf = [ttZ[1][-2], tttt[1][-2], ttW[1][-2]]
fd = pd.DataFrame({r'Background': name,
    'Cross Section in Paper (fb)': n1,
                   'My Result with jet addition': n2,
                   'My Result without jet addition': n3,
                   'Uncertainty': uf
                   })
fd

,Background,Cross Section in Paper (fb),My Result with jet addition,My Result without jet addition,Uncertainty
0,ttZ,0.135,0.122112,0.113360,0.000304
1,4t,0.105,0.037159,0.037159,0.000759
2,ttW,0.011,0.010619,0.003727,0.000185
